In [1]:
import json
from tronpy import Tron
from tronpy.providers import HTTPProvider
from tronpy.keys import PrivateKey
import time

from tronpy.contract import Contract

In [2]:
client = Tron(network="shasta")

In [3]:
try:
    # Check connection by getting the latest block number
    latest_block = client.get_latest_block_number()
    print(f"Successfully connected to the TRON node at shasta")
    print(f"Latest block number: {latest_block}")
except Exception as e:
    print(f"Connection to TRON node failed. Error: {str(e)}")
    print("Please check the URL and try again.")

Successfully connected to the TRON node at shasta
Latest block number: 48343277


In [4]:

# get private data from ..\neuranft-contract\.env
with open('../neuranft-contract/.env', 'r') as f:
    data = f.readlines()
    for line in data:
        if 'PRIVATE_KEY_SHASTA' in line:
            private_key = line.split('=')[1].strip()

In [5]:
#collection json
with open(r'..\neuranft-contract\build\contracts\CollectionContract.json') as f:
    collection_json = json.load(f)

with open(r'..\neuranft-contract\build\contracts\NFTContract.json') as f:
    nft_json = json.load(f)


with open(r'..\neuranft-contract\build\contracts\NFTAccessControl.json') as f:
    NFTAccessControl_json = json.load(f)


with open(r'..\neuranft-contract\build\contracts\NFTMetadata.json') as f:
    NFTMetadata_json = json.load(f)
    
with open(r'..\neuranft-contract\build\contractAddresses\shasta_addresses.json') as f:
    address_json= json.load(f)

In [6]:
address_json

{'MasterAccessControl': '41958b175bf50997a116901d2601b92484d5a4510f',
 'NFTAccessControl': '418d8207694330b7a66fa191b501533f2d0247dd3d',
 'NFTMetadata': '41d16d16bce1b9238f110466e31c740042babf5432',
 'NFTContract': '4106fc009e483b37edbbcb392df1115cb4376e50c3',
 'CollectionContract': '4169bd221187375bbbebcf300b6b8a9f6cacca9bbd'}

In [7]:
account_address = PrivateKey(bytes.fromhex(private_key)).public_key.to_base58check_address()

In [9]:
from tronpy.abi import trx_abi

import base58


def hex_to_base58(hex_string):
    if hex_string[:2] in ["0x", "0X"]:
        hex_string = "41" + hex_string[2:]
    bytes_str = bytes.fromhex(hex_string)
    base58_str = base58.b58encode_check(bytes_str)
    return base58_str.decode("UTF-8")

In [ ]:
hex_to_base58(address_json["CollectionContract"])

In [ ]:
client = Tron(network="shasta")
account = client.get_account(PrivateKey(bytes.fromhex(private_key)).public_key.to_base58check_address())



In [29]:
collection_contract         = client.get_contract(address_json["CollectionContract"])
nft_contract                = client.get_contract(address_json["NFTContract"])
nft_metadata_contract       = client.get_contract(address_json["NFTMetadata"])
nft_access_control_contract = client.get_contract(address_json["NFTAccessControl"])


collection_contract.abi     = collection_json["abi"]
nft_contract.abi            = nft_json["abi"]
nft_metadata_contract.abi   = NFTMetadata_json["abi"]
nft_access_control_contract.abi = NFTAccessControl_json["abi"]

In [30]:

# Connect to the TRON network
client = Tron(network="shasta")

tron_provider_url = 'shasta'

try:
    # Check connection by getting the latest block number
    latest_block = client.get_latest_block_number()
    print(f"Successfully connected to the TRON node at {tron_provider_url}")
    print(f"Latest block number: {latest_block}")
except Exception as e:
    print(f"Connection to TRON node failed. Error: {str(e)}")
    print("Please check the URL and try again.")





Successfully connected to the TRON node at shasta
Latest block number: 48345086


In [32]:
hex_to_base58(address_json["CollectionContract"])

'TKcJXQ32KMZhwZCccb6XQ51VJ79Yna4JhX'

In [34]:
with open('./data/all_collections.json') as f:
    collections = json.load(f)

In [69]:
def write_to_contract(contract, function_name: str, 
                      arguments: list, 
                      private_key: str, 
                      fee_limit = 1000000000):
    """
    Write data to a contract function on the TRON network
    :param contract: The contract object
    :param function_name: The name of the function to call
    :param arguments: The arguments to pass to the function
    :param private_key: The private key of the account calling the function
    :param tron_provider_url: The URL of the TRON node
    :param fee_limit: The maximum energy cost you're willing to pay (in SUN)
    :return: The transaction result
    """


    # Create a private key object
    priv_key = PrivateKey(bytes.fromhex(private_key))

    # Get the corresponding account
    account = client.get_account(priv_key.public_key.to_base58check_address())

    # Build and sign the transaction
    try:
        # print(" starting 1.1 building transaction")
        txn = (
            contract.functions[function_name](*arguments)
            .with_owner(account_address)
            .fee_limit(fee_limit)
            .build()
            .sign(priv_key)
        )

        # print(" starting 1.2 broadcasting transaction")
        # Broadcast the transaction
        result = txn.broadcast().wait()

        return result
    except Exception as e:
        print(f"Error while building, signing, or broadcasting the transaction: {str(e)}")
        return None

# Collection

In [64]:
def create_collection(collection):
    function_name = "createCollection"
    arguments = [collection['name'], collection['contextWindow'], collection['model'], collection['image'], collection['description']]
    
    try:

        
        result = write_to_contract(collection_contract, function_name, arguments, private_key)
        
        # print ("Result for create collection ", collection['name'], " is ", result['receipt']['result'])
        
        return result
        
        # print(result)
        
        # print("starting 2")
        
        # # In TRON, we need to check the events differently
        # for log in result['receipt']['result']:
        #     if log['name'] == 'CollectionCreated':
        #         collection_id = log['parameter']['collectionId']
        #         return collection_id
        
        # print("CollectionCreated event not found in transaction logs")
        # return None
    except Exception as e:
        print(f"Error creating collection {collection['name']}: {str(e)}")
        return None

def getAllCollections():
    try:
        
        total_collections = collection_contract.functions.getAllCollections()
        
        all_collections = []
        counter = 1
        for collection_data in total_collections:
            # collection_data = collection_contract.functions.getCollectionMetadata(i)
            i = counter
            counter += 1
            formatted_collection = {
                "id": i,
                "name": collection_data[0],
                "contextWindow": collection_data[1],
                "model": collection_data[2],
                "image": collection_data[3],
                "description": collection_data[4],
                "creator": collection_data[5],
                "date": collection_data[6],
                "owner": collection_data[7],
                "collectionaddress": f"#{i}",
                # "noOfNFTs": collection_contract.functions.getCollectionNFTCount(i)
            }
            all_collections.append(formatted_collection)

        return all_collections
    except Exception as e:
        print(f"Error getting all collections: {str(e)}")
        return None

def getAllCollections_by_address(address):
    try:
        all_collections = getAllCollections()
        return [collection for collection in all_collections if collection['owner'] == address]
    except Exception as e:
        print(f"Error getting collections for address {address}: {str(e)}")
        return None

def get_collection_details_by_id(collection_id):
    try:
        metadata = collection_contract.functions.getCollectionMetadata(collection_id)
        nft_count = collection_contract.functions.getCollectionNFTCount(collection_id)
        owner = collection_contract.functions.getCollectionOwner(collection_id)
        unique_holders = collection_contract.functions.getCollectionUniqueHolders(collection_id)

        collection_details = {
            "id": collection_id,
            "name": metadata[0],
            "contextWindow": metadata[1],
            "baseModel": metadata[2],
            "image": metadata[3],
            "description": metadata[4],
            "creator": metadata[5],
            "dateCreated": metadata[6],
            "owner": owner,
            "collectionaddress": f"#{collection_id}",
            "noOfNFTs": nft_count,
            "uniqueHolders": unique_holders
        }

        return collection_details
    except Exception as e:
        print(f"Error getting details for collection ID {collection_id}: {str(e)}")
        return None
    
def details_to_json(collection_id):
    
    function_name = "createCollection"
    arguments = [collection['name'], collection['contextWindow'], collection['model'], collection['image'], collection['description']]
    
    try:
        result = write_to_contract(collection_contract, function_name, arguments, private_key)        
        return result

    except Exception as e:
        print(f"Error creating collection {collection['name']}: {str(e)}")
        return None

In [ ]:
results = []
for collection in collections["collections"][3:]:
    results += [create_collection(collection)]


In [66]:
getAllCollections_by_address("TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka")

[{'id': 1,
  'name': 'Neural Mint Hub',
  'contextWindow': 4096,
  'model': 'Llama 3.1 70B',
  'image': 'http://localhost:5500/image/temp7.png',
  'description': 'Discover the fusion of neural networks and blockchain at the Neural Mint Hub. This collection offers unique AI-generated NFTs crafted by advanced neural models.',
  'creator': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'date': 1728676872,
  'owner': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'collectionaddress': '#1'},
 {'id': 2,
  'name': 'Quantum Rewards',
  'contextWindow': 2048,
  'model': 'Llama 3.1',
  'image': 'http://localhost:5500/image/temp2.png',
  'description': 'Unlock quantum leaps in rewards with our AI-enhanced NFT collection. Each piece represents the convergence of quantum computing concepts and digital art.',
  'creator': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'date': 1728677826,
  'owner': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'collectionaddress': '#2'},
 {'id': 3,
  'name': 'AI Profile Architect',
  'context

In [82]:
getAllCollections()

[{'id': 1,
  'name': 'Neural Mint Hub',
  'contextWindow': 4096,
  'model': 'Llama 3.1 70B',
  'image': 'http://localhost:5500/image/temp7.png',
  'description': 'Discover the fusion of neural networks and blockchain at the Neural Mint Hub. This collection offers unique AI-generated NFTs crafted by advanced neural models.',
  'creator': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'date': 1728676872,
  'owner': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'collectionaddress': '#1'},
 {'id': 2,
  'name': 'Quantum Rewards',
  'contextWindow': 2048,
  'model': 'Llama 3.1',
  'image': 'http://localhost:5500/image/temp2.png',
  'description': 'Unlock quantum leaps in rewards with our AI-enhanced NFT collection. Each piece represents the convergence of quantum computing concepts and digital art.',
  'creator': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'date': 1728677826,
  'owner': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'collectionaddress': '#2'},
 {'id': 3,
  'name': 'AI Profile Architect',
  'context

In [46]:
collection_contract.functions.getAllCollections()

(('Neural Mint Hub',
  4096,
  'Llama 3.1 70B',
  'http://localhost:5500/image/temp7.png',
  'Discover the fusion of neural networks and blockchain at the Neural Mint Hub. This collection offers unique AI-generated NFTs crafted by advanced neural models.',
  'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  1728676872,
  'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka'),
 ('Quantum Rewards',
  2048,
  'Llama 3.1',
  'http://localhost:5500/image/temp2.png',
  'Unlock quantum leaps in rewards with our AI-enhanced NFT collection. Each piece represents the convergence of quantum computing concepts and digital art.',
  'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  1728677826,
  'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka'),
 ('AI Profile Architect',
  8192,
  'Llama 3.1 70B',
  'http://localhost:5500/image/temp3.png',
  'Design your digital identity with AI Profile Architect. This collection offers AI-crafted NFTs that redefine personal expression in the digital realm.',
  'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  1728681363,
 

In [54]:
get_collection_details_by_id(1)

{'id': 1,
 'name': 'Neural Mint Hub',
 'contextWindow': 4096,
 'baseModel': 'Llama 3.1 70B',
 'image': 'http://localhost:5500/image/temp7.png',
 'description': 'Discover the fusion of neural networks and blockchain at the Neural Mint Hub. This collection offers unique AI-generated NFTs crafted by advanced neural models.',
 'creator': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
 'dateCreated': 1728676872,
 'owner': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
 'collectionaddress': '#1',
 'noOfNFTs': 0,
 'uniqueHolders': 0}

# NFT Cretion

In [84]:

def create_NFT(nft_data):
    

        
    # write to contract
    
    
    function_name = "createNFT"
    
    _collectionId = nft_data['collection_id']
    _name = nft_data['name']
    level_of_ownership = 6
    
    
    arguments = [_collectionId, _name, level_of_ownership]
    key = private_key
    
    
    try :
        result = write_to_contract(nft_contract, function_name, arguments, key)
        print("NFT created : ", nft_data['name'])
        return result
    except Exception as e:
        print("Error creating nft : ", collection['name'])
        print("Error : ", e)    

def create_metadata(nft_data):
    

        
    # write to contract
    
    
    function_name = "createMetadata"
    
    _collectionId = nft_data['collection_id']
    _nftId = nft_data['id']
    
    _metadata = [
        nft_data['image'],
        nft_data['model'],
        "http//localhost:5500/data",
        "http//localhost:5500/rag",
        "localhost:5500/fineTuneData",
        nft_data['description']
    ]
    
    
    
    arguments = [_collectionId, _nftId, _metadata]
    key = private_key
    
    
    try :
        result = write_to_contract(nft_metadata_contract, function_name, arguments, key)
        print("NFT metadata created : ", nft_data['name'])
        return result
        
    except Exception as e:
        print("Error creating nft metadata: ", nft_data['name'])
        
def provide_access(nft_data):
    
    
    if "accessList" not in nft_data:
        return []
        
    _access_list =  nft_data['accessList']
    
    
    function_name = "grantAccess"
    
    _collectionId = nft_data['collection_id']
    _nftId = nft_data['id']
    
    key = private_key
    
    results = []
    
    print("Providing access for nft : ", nft_data['name'])
    
    for access in _access_list:
        
        _user = access['address']
        _level = access['accessLevel']
        
        arguments = [_collectionId, _nftId, _user, _level]
        
        try :
            result = write_to_contract(nft_access_control_contract, function_name, arguments, key)
            results.append(result)
            
            print("\t\t Access granted to user : ", _user , " with access level : ", _level)
            
        except Exception as e:
            print("\t\t Error granding user : ", _user) 
            print("Error : ", e)    
        
    return results
        
        
        
def burnNFT(collection_id, nft_id):
    
    function_name = "burnNFT"
    arguments = [collection_id, nft_id]
    
    try:
        result = write_to_contract(nft_contract, function_name, arguments, private_key)
        print("NFT burned : ", nft_id, " from collection : ", collection_id)        
        return result

    except Exception as e:
        print(f"Error creating collection {collection['name']}: {str(e)}")
        return None
        


In [71]:
with open('./data/all_nft_tron.json') as f:
    all_nft_tron = json.load(f)

In [72]:
all_nft_tron

{'nfts': [{'id': 1,
   'name': 'Cognitive Llama',
   'collection': 'Neural Mint Hub',
   'collection_id': 1,
   'image': 'http://localhost:5500/image/temp11.jpg',
   'model': 'Llama 3.1',
   'attributes': [{'trait_type': 'Model', 'value': 'Llama 3.1'},
    {'trait_type': 'Context Window', 'value': '4096 tokens'},
    {'trait_type': 'Total Access', 'value': '1'},
    {'trait_type': 'Collection', 'value': 'Neural Mint Hub'}],
   'accessList': [{'address': 'TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr',
     'accessLevel': 6},
    {'address': 'TThwHjyzx2E1TNN4GdSoaHeCRJVi3EuKS7', 'accessLevel': 2},
    {'address': 'TJfqnAUwX2ViGTP1J81itXWhuJhUYa8RwK', 'accessLevel': 2},
    {'address': 'TVh65XCYkerBch3WtgY9kwLqrtH64MhiFe', 'accessLevel': 3},
    {'address': 'TMVQGm1qAQYVdetCeGRRkTWYYrLXuHK2HC', 'accessLevel': 1}],
   'tokenId': '1234',
   'tokenStandard': 'ERC-721',
   'owner': 'TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr',
   'chain': 'Ethereum',
   'description': 'Cognitive Llama is an advanced AI-driven 

In [73]:
for nft in all_nft_tron["nfts"]:
    nft_id = create_NFT(nft)
    # nft['id'] = nft_id
    # print(f"NFT ID: {nft_id}")

NFT created :  Cognitive Llama
NFT created :  Cyborg Bored Ape
NFT created :  Thinker Primate
NFT created :  MankiBean
NFT created :  based punk
NFT created :  Dark Llama
NFT created :  Tron Cells
NFT created :  Corrupted Cyborg
NFT created :  Skynet


In [74]:
for nft in all_nft_tron["nfts"]:
    nft_id = create_metadata(nft)
    # nft['id'] = nft_id
    # print(f"NFT ID: {nft_id}")

NFT metadata created :  Cognitive Llama
NFT metadata created :  Cyborg Bored Ape
NFT metadata created :  Thinker Primate
NFT metadata created :  MankiBean
NFT metadata created :  based punk
NFT metadata created :  Dark Llama
NFT metadata created :  Tron Cells
NFT metadata created :  Corrupted Cyborg
NFT metadata created :  Skynet


In [77]:
for nft in all_nft_tron["nfts"]:
    nft_id = provide_access(nft)
    # nft['id'] = nft_id
    # print(f"NFT ID: {nft_id}")

Providing access for nft :  Cognitive Llama
		 Access granted to user :  TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr  with access level :  6
		 Access granted to user :  TThwHjyzx2E1TNN4GdSoaHeCRJVi3EuKS7  with access level :  2
		 Access granted to user :  TJfqnAUwX2ViGTP1J81itXWhuJhUYa8RwK  with access level :  2
		 Access granted to user :  TVh65XCYkerBch3WtgY9kwLqrtH64MhiFe  with access level :  3
		 Access granted to user :  TMVQGm1qAQYVdetCeGRRkTWYYrLXuHK2HC  with access level :  1
Providing access for nft :  Cyborg Bored Ape
		 Access granted to user :  TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr  with access level :  1
Providing access for nft :  Thinker Primate
		 Access granted to user :  TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr  with access level :  1
		 Access granted to user :  TJfqnAUwX2ViGTP1J81itXWhuJhUYa8RwK  with access level :  2
Providing access for nft :  MankiBean
		 Access granted to user :  TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr  with access level :  1
Providing access for nft :  based pun

In [85]:
# burning nfts
for i in range(8,15):
    burnNFT(1, i)
    
    

NFT burned :  8  from collection :  1
NFT burned :  9  from collection :  1
NFT burned :  10  from collection :  1
NFT burned :  11  from collection :  1
NFT burned :  12  from collection :  1
NFT burned :  13  from collection :  1
NFT burned :  14  from collection :  1


In [78]:
def all_nft_information(nft_id, collection_id):
    try:
        nft_info = nft_contract.functions.getNFTInfo(collection_id, nft_id)
        metadata = nft_metadata_contract.functions.getMetadata(collection_id, nft_id)
        
        return {
            "id": nft_id,
            "collectionId": collection_id,
            "levelOfOwnership": nft_info[0],
            "name": nft_info[1],
            "creator": nft_info[2],
            "creationDate": nft_info[3],
            "owner": nft_info[4],
            "metadata": {
                "image": metadata[0],
                "baseModel": metadata[1],
                "data": metadata[2],
                "rag": metadata[3],
                "fineTuneData": metadata[4],
                "description": metadata[5]
            }
        }
    except Exception as e:
        print(f"Error getting information for NFT ID {nft_id} in collection {collection_id}: {e}")
        return None

def all_nft_of_a_collection(collection_id):
    try:
        nft_count = nft_contract.functions.getCollectionNFTCount(collection_id)
        nft_ids = nft_contract.functions.getCollectionNFTs(collection_id)
        
        nfts = []
        for nft_id in nft_ids:
            nft_info = all_nft_information(nft_id, collection_id)
            if nft_info:
                nfts.append(nft_info)
        
        return nfts
    except Exception as e:
        print(f"Error getting NFTs for collection {collection_id}: {e}")
        return None

def all_access_levels_of_a_collection_nft(collection_id, nft_id):
    try:
        users_access = nft_access_control_contract.functions.getAllUsersAccessForNFT(collection_id, nft_id)
        
        access_levels = []
        for user_access in users_access:
            access_levels.append({
                "user": user_access[0],
                "accessLevel": user_access[1]
            })
        
        return access_levels
    except Exception as e:
        print(f"Error getting access levels for NFT ID {nft_id} in collection {collection_id}: {e}")
        return None

def all_nfts_own_or_have_access_by_user(user_address):
    try:
        user_access_entries = nft_access_control_contract.functions.getAllAccessForUser(user_address)
        
        nfts = []
        for entry in user_access_entries:
            collection_id, nft_id, access_level = entry
            nft_info = all_nft_information(nft_id, collection_id)
            if nft_info:
                nft_info['accessLevel'] = access_level
                nfts.append(nft_info)
        
        return nfts
    except Exception as e:
        print(f"Error getting NFTs for user {user_address}: {e}")
        return None


In [79]:
all_nft_information(1, 1)

{'id': 1,
 'collectionId': 1,
 'levelOfOwnership': 6,
 'name': 'Cognitive Llama',
 'creator': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
 'creationDate': 1728689829,
 'owner': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
 'metadata': {'image': 'http://localhost:5500/image/temp11.jpg',
  'baseModel': 'Llama 3.1',
  'data': 'http//localhost:5500/data',
  'rag': 'http//localhost:5500/rag',
  'fineTuneData': 'localhost:5500/fineTuneData',
  'description': 'Cognitive Llama is an advanced AI-driven bot, utilizing the power of Llama 3.1 with multi-level reasoning and cognition. Part of the Neural Mint Hub collection.'}}

In [86]:
all_nft_of_a_collection(1)

[{'id': 1,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Cognitive Llama',
  'creator': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'creationDate': 1728689829,
  'owner': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'metadata': {'image': 'http://localhost:5500/image/temp11.jpg',
   'baseModel': 'Llama 3.1',
   'data': 'http//localhost:5500/data',
   'rag': 'http//localhost:5500/rag',
   'fineTuneData': 'localhost:5500/fineTuneData',
   'description': 'Cognitive Llama is an advanced AI-driven bot, utilizing the power of Llama 3.1 with multi-level reasoning and cognition. Part of the Neural Mint Hub collection.'}},
 {'id': 2,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Cyborg Bored Ape',
  'creator': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'creationDate': 1728689835,
  'owner': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka',
  'metadata': {'image': 'http://localhost:5500/image/temp10.jpg',
   'baseModel': 'GPT-4',
   'data': 'http//localhost:5500/data',
   'rag': 'http//localhost:55

In [81]:
all_access_levels_of_a_collection_nft(1,1)

[{'user': 'TFJGXg1C962aDqSyvdSZXZ9am2ETqwNQka', 'accessLevel': 6},
 {'user': 'TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr', 'accessLevel': 6},
 {'user': 'TThwHjyzx2E1TNN4GdSoaHeCRJVi3EuKS7', 'accessLevel': 2},
 {'user': 'TJfqnAUwX2ViGTP1J81itXWhuJhUYa8RwK', 'accessLevel': 2},
 {'user': 'TVh65XCYkerBch3WtgY9kwLqrtH64MhiFe', 'accessLevel': 3},
 {'user': 'TMVQGm1qAQYVdetCeGRRkTWYYrLXuHK2HC', 'accessLevel': 1}]

In [ ]:
all_nfts_own_or_have_access_by_user("0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd")
